#TRAFx Data Manipulation and Plotting
This python script is for manipulating TRAFx data from the 'Raw Export' file type which comes with two un-named columns but are effectively (Date | Count) 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 23 13:56:41 2021

@author: NoahCreany
"""
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import io
import datetime
import xlrd
import os

from functools import reduce
#stats, figs, and plots
from scipy import stats
import seaborn as sns
from scipy.interpolate import make_interp_spline, BSpline

import matplotlib.ticker as mtick


You'll have to upload the file to Google Drive to access it here


In [ ]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


Create a new dataframe for each counter with the path from the Google Drive and 
infer dates from TRAFx timestamp


In [ ]:
counter1 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/May23-Oct23 HWY traffic data CRMO.csv', parse_dates=[['DATADTTM','HOUR']])
counter1['Date'] = pd.to_datetime(counter1.DATADTTM_HOUR, format="%m/%d/%Y %H")
counter1.replace=[{'DATA': 'HighwayData'} ]

counter2 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- Fee Booth Vehicle[entrance] RAW.csv')
counter2.columns=['Date', 'FeeBooth']
counter2['Date'] = pd.to_datetime(counter2.Date)

counter3 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- One Way Start RAW.csv')
counter3.columns=['Date', 'OneWayStart']
counter3['Date'] = pd.to_datetime(counter3.Date)

counter4 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- One Way End  RAW.csv')
counter4.columns=['Date', 'OneWayEnd']
counter4['Date'] = pd.to_datetime(counter4.Date)

counter5 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- Devils Orchard Vehicle[entrance] RAW.csv')
counter5.columns=['Date', 'DevilsOrchardVehicle']
counter5['Date'] = pd.to_datetime(counter5.Date)

counter6 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- North Crater Trail RAW.csv')
counter6.columns=['Date', 'NorthCrater']
counter6['Date'] = pd.to_datetime(counter6.Date)

counter7 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- Devils Orchard Trail RAW.csv')
counter7.columns=['Date', 'DevilsOrchardTrail']
counter7['Date'] = pd.to_datetime(counter7.Date)

counter8 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- Tree Molds Trail RAW.csv')
counter8.columns=['Date', 'TreeMolds']
counter8['Date'] = pd.to_datetime(counter8.Date)

counter9 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- Broken Top Trail RAW.csv')
counter9.columns=['Date', 'BrokenTop']
counter9['Date'] = pd.to_datetime(counter9.Date)

counter10 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- Wilderness Trail RAW.csv')
counter10.columns=['Date', 'WildernessTrail']
counter10['Date'] = pd.to_datetime(counter10.Date)

counter11 = pd.read_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO21- Caves2 RAW.csv')
counter11.columns=['Date', 'Caves']
counter11['Date'] = pd.to_datetime(counter11.Date)

Merge Dataframes together into a single CSV

In [ ]:
data_frames=[counter1,counter2,counter3,counter4,counter5,counter6,counter7,counter8,counter9,counter10,counter11]

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['Date'],
how='outer'), data_frames).fillna(999)

#df_merged = pd.concat(data_frames, join='outer', axis=1).fillna(999)


df_merged.to_csv('/content/gdrive/MyDrive/CRMO_Data/Merged.csv')

Identify and Interpolate Outliers

In [ ]:
trail_df = pd.read_excel('/content/gdrive/MyDrive/CRMO_Data/CRMO_Combined_Counter_Dataset.xlsx')
Trails = ['FeeBooth','OneWayStart','OneWayEnd','DevilsOrchardVehicle','NorthCrater','DevilsOrchardTrail','TreeMolds','BrokenTop','WildernessTrail','Caves']


In [ ]:
trail_df.head()

,Date/Time,Date,Time,Highway,FeeBooth,OneWayStart,OneWayEnd,DevilsOrchardVehicle,NorthCrater,DevilsOrchardTrail,TreeMolds,BrokenTop,WildernessTrail,Caves,VCLot,DevilsOrchardLot,NorthCraterLot,InfernoConeLot,SnowConesLot,WildernessLot,CavesLot,DayofWeek,Hour,Mean,Std,Z,FeeBoothOutlier,FeeBoothCorrectedCount,OneWayStartOutlier,OneWayStartCorrectedCount,OneWayEndOutlier,OneWayEndCorrectedCount,DevilsOrchardVehicleOutlier,DevilsOrchardVehicleCorrectedCount,NorthCraterOutlier,NorthCraterCorrectedCount,DevilsOrchardTrailOutlier,DevilsOrchardTrailCorrectedCount,TreeMoldsOutlier,TreeMoldsCorrectedCount,BrokenTopOutlier,BrokenTopCorrectedCount,WildernessTrailOutlier,WildernessTrailCorrectedCount,CavesOutlier,CavesCorrectedCount
0,2021-05-26 00:00:00,2021-05-26,00:00:00,5,0.0,999.0,999.0,999.0,999.0,0.0,0.000000,0.0,0.0,999.0,999,999,999,999,999,999,999,Wednesday,0,882.417587,314.99084,0.370114,False,0.0,False,999.0,True,NaN,True,NaN,True,NaN,False,0.0,False,0.0,False,0.0,False,0.0,False,999.0
1,2021-05-26 01:00:00,2021-05-26,01:00:00,4,0.0,999.0,999.0,999.0,999.0,0.0,4.056605,0.0,0.0,999.0,999,999,999,999,999,999,999,Wednesday,0,882.417587,314.99084,0.370114,False,0.0,False,999.0,True,NaN,True,NaN,True,NaN,False,0.0,False,4.0,False,0.0,False,0.0,False,999.0
2,2021-05-26 02:00:00,2021-05-26,02:00:00,4,0.0,999.0,999.0,999.0,999.0,0.0,0.000000,0.0,0.0,999.0,999,999,999,999,999,999,999,Wednesday,0,882.417587,314.99084,0.370114,False,0.0,False,999.0,True,NaN,True,NaN,True,NaN,False,0.0,False,0.0,False,0.0,False,0.0,False,999.0
3,2021-05-26 03:00:00,2021-05-26,03:00:00,2,0.0,999.0,999.0,999.0,999.0,0.0,0.000000,0.0,0.0,999.0,999,999,999,999,999,999,999,Wednesday,0,882.417587,314.99084,0.370114,False,0.0,False,999.0,True,NaN,True,NaN,True,NaN,False,0.0,False,0.0,False,0.0,False,0.0,False,999.0
4,2021-05-26 04:00:00,2021-05-26,04:00:00,7,0.0,999.0,999.0,999.0,999.0,0.0,0.000000,0.0,0.0,999.0,999,999,999,999,999,999,999,Wednesday,0,882.417587,314.99084,0.370114,False,0.0,False,999.0,True,NaN,True,NaN,True,NaN,False,0.0,False,0.0,False,0.0,False,0.0,False,999.0


In [ ]:
#Outlier Function:
def outlier(df, trailname):
  df.replace({999:np.nan}, inplace=True)
  dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
  df['DayofWeek']=df['Date'].dt.dayofweek.map(dayOfWeek)
  df['Hour']=df['Date'].dt.hour
  df['Mean']=df.groupby(['DayofWeek','Hour'])[trailname].transform('mean')
  df['Std']=df.groupby(['DayofWeek','Hour'])[trailname].transform('std')
  df['Z']=(df[trailname]-df.Mean)/df.Std
  df[str(trailname + 'Outlier')]= df.Z.abs()>3
  df[str(trailname + 'CorrectedCount')]=df[trailname].mask(df[str(trailname + 'Outlier')]).interpolate('from_derivatives').round()
  df=df[['Date','DayofWeek','Hour',trailname,str(trailname + 'CorrectedCount'),str(trailname + 'Outlier')]]
  return df

for i in Trails:  
  outlier(trail_df, i)

trail_df.to_csv('/content/gdrive/MyDrive/CRMO_Data/CRMO_Combined_Counter_Dataset.csv')

In [ ]:
# =============================================================================
# We can then split the data by mean counts by Day of the Week:
# =============================================================================

GroupedByDayOfWeek = Data.groupby(Data['Date'].dt.day_name())['Count'].mean()

DayofWeek = pd.DataFrame(GroupedByDayOfWeek)
DayofWeek['Day']=DayofWeek.index

#Lets Visualize:
sns.set(style = 'darkgrid') #Background style
plt.figure(figsize = (10,7)) # Figure size
#Create Font dictionaries for plot fonts
Titlefont = {'family': 'serif', 
        'color':  'black',
        'weight': 'normal',
        'size': 16,}
#set fomt size for axes
AxesFont= {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,}
    
ax = sns.barplot(
        x = 'Day',
        y = 'Count',
        data = DayofWeek,
        order = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'],
        color='#69b3a2')
plt.xlabel('Day of Week', fontdict = AxesFont)
plt.ylabel('Mean # of Visitors', fontdict = AxesFont)
plt.title('Average Counts for Counter Name by Day of Week', fontdict = Titlefont)
plt.show()

In [ ]:
# =============================================================================
# Or perhaps we're interested in the Hour of the Day:
# =============================================================================
GroupedByHour = Data.groupby(Data['Date'].dt.hour)['Count'].mean()
HourOfDay = pd.DataFrame(GroupedByHour)
HourOfDay['Hour'] = HourOfDay.index

sns.set(style = 'darkgrid') #Background style
plt.figure(figsize = (10,7)) # Figure size
#Create Font dictionaries for plot fonts
Titlefont = {'family': 'serif', 
        'color':  'black',
        'weight': 'normal',
        'size': 16,}
#set fomt size for axes
AxesFont= {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,}
    
ax = sns.barplot(
        x = 'Hour',
        y = 'Count',
        data = HourOfDay,
        color='#69b3a2')
plt.xlabel('Hour of Day', fontdict = AxesFont)
plt.ylabel('Mean # of Visitors', fontdict = AxesFont)
plt.title('Average Hourly Counts for Counter Name', fontdict = Titlefont)
plt.show()

In [ ]:

# =============================================================================
# Maybe hourly counts by Weekday vs Weekend:
# =============================================================================
# This first step is just to create a variable to determine which days are weekdays vs weekends
Data['Weekday'] = ((pd.DatetimeIndex(Data.Date).dayofweek) // 5 == 1).astype(float)

GroupedByDayHour = Data.groupby([Data['Weekday'], Data['Date'].dt.hour])['Count'].mean()
Weekday_vs_Weekend_Hourly = GroupedByDayHour.to_frame().reset_index()
Weekday_vs_Weekend_Hourly.columns = ['Weekday','Hour','Count']

sns.set(style = 'darkgrid') #Background style
plt.figure(figsize = (10,7)) # Figure size
#Create Font dictionaries for plot fonts
Titlefont = {'family': 'serif', 
        'color':  'black',
        'weight': 'normal',
        'size': 16,}
#set fomt size for axes
AxesFont= {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,}
sns.set(style = 'darkgrid') #Background style

plt.figure(figsize = (10,7)) # Figure size

#Create Font dictionaries for plot fonts
Titlefont = {'family': 'serif', 
        'color':  'black',
        'weight': 'normal',
        'size': 16,}
#set fomt size for axes
AxesFont= {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,}
colors = ["#69b3a2", "#4374B3"]
sns.set_palette(sns.color_palette(colors))
    
ax = sns.barplot(
        x = 'Hour',
        y = 'Count',
        data = Weekday_vs_Weekend_Hourly,
        hue = 'Weekday')
plt.xlabel('Hour of Day', fontdict = AxesFont)
plt.ylabel('Mean # of Visitors', fontdict = AxesFont)
plt.title('Weekday vs Weekend Hourly Counts for Counter Name', fontdict = Titlefont)
plt.legend(['Weekend','Weekday'])
plt.show()